In [13]:
import pandas as pd
from collections import Counter
import jieba

In [2]:
data = pd.read_csv('../feature/data_cutted.csv')

In [3]:
data.head()

,id,content,source
0,yhM4ivIm,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train
1,OeJidjGX,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train
2,4wWp3dGK,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train
3,T0senQto,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train
4,1jAboq7C,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['id'] = le.fit_transform(data['id'])

#list(le.inverse_transform([2, 2, 1]))

In [5]:
data.head()

,id,content,source
0,98495,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train
1,40279,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train
2,8195,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train
3,47349,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train
4,2839,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train


In [6]:
def get_words(txt):
    txt = jieba.cut(txt)  
    c = Counter()  
    for x in txt:  
        if len(x)>1 and x != '\r\n':  
            c[x] += 1
    return c

In [7]:
data['vector_content'] = data['content'].apply(lambda x: get_words(str(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\kuhung\AppData\Local\Temp\jieba.cache
Loading model cost 1.373 seconds.
Prefix dict has been built succesfully.


In [8]:
data.head()

,id,content,source,vector_content
0,98495,ft ir 产物 结构 进行 分析 扫描电镜 查看 喷洒 抑尘剂 煤粉 固化 表面 形貌,train,"{'ft': 1, 'ir': 1, '产物': 1, '结构': 1, '进行': 1, ..."
1,40279,方法 利用 硅胶 ods 及大孔 树脂 hp 多种 色谱 方法 对密 花石 豆兰 乙醇 提取...,train,"{'方法': 2, '利用': 1, '硅胶': 1, 'ods': 1, '及大孔': 1..."
2,8195,聚烯烃 技术 快速 发展 市场占有率 不断 提高 关键 科学技术 发展 引起 学术界 产业界...,train,"{'聚烯烃': 1, '技术': 1, '快速': 1, '发展': 2, '市场占有率':..."
3,47349,猪脑 心肌炎 病毒 emcv gxlc 基因组 序列 设计 一对 特异性 引物 应用 rt...,train,"{'猪脑': 1, '心肌炎': 1, '病毒': 1, 'emcv': 3, 'gxlc'..."
4,2839,结果表明 实生苗 造林 萌芽 更新 杉木 林分 树高 胸径 材积 生长 插条 造林 实生苗 ...,train,"{'结果表明': 1, '实生苗': 2, '造林': 3, '萌芽': 2, '更新': ..."


In [9]:
def get_cosine_new(vec1, vec2):
    try:
        present_count = 0
        total_count = 0
        for word,count in vec1.items():
            total_count += count
            if word in vec2:
                present_count += count
        return present_count/max(total_count,1)
    except:
        return 0.0

In [16]:
def find_relation(inp_df):
    relation_df = []
    train = inp_df.loc[data['source']=='train'].reset_index()
    test = inp_df.loc[data['source']=='test'].reset_index()
    
    for i in range(len(test)):
        #print (i)
        for j in range(len(train)):
            rel_df = dict()
            try:
                rel_df["cosine_abstract"] = get_cosine_new(test['vector_content'][i],train['vector_content'][j])
            except:
                rel_df["cosine_abstract"] = 0.0
            
            #print (rel_df)
            if rel_df["cosine_abstract"] > 0.0:
                rel_df["id"] = test["id"][i]
                rel_df["related_id"] = train["id"][j]
                relation_df.append(rel_df)
            else:
                pass
    final_df = pd.DataFrame(relation_df)
    return final_df

In [17]:
frequency_vec = find_relation(data)

Wall time: 0 ns
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
27

In [22]:
frequency_vec['id'] = le.inverse_transform(frequency_vec['id'])
frequency_vec['related_id'] = le.inverse_transform(frequency_vec['related_id'])

KeyboardInterrupt: 

In [25]:
frequency_vec.to_csv('../feature/cosine_abstract.csv',index=False)

In [89]:
frequency_vec = pd.read_csv('../feature/cosine_abstract.csv')

In [90]:
print(frequency_vec.shape)

(20636631, 3)


In [91]:
print(frequency_vec.describe())

       cosine_abstract
count     2.063663e+07
mean      7.422552e-02
std       5.079674e-02
min       5.494505e-03
25%       3.846154e-02
50%       6.060606e-02
75%       9.375000e-02
max       1.000000e+00


In [93]:
#frequency_pivot.replace({'﻿mhY5opF':'?mhY5opF4'},inplace=True) 
frequency_pivot = frequency_vec.set_index(['id','related_id']).unstack()

In [126]:
result = frequency_pivot.idxmax(axis = 1).apply(lambda x : str(x)[21:29])

In [136]:
import numpy as np
result_pro = np.max(frequency_pivot,axis=1)

In [137]:
frequency_pivot.index

Index(['02dDNCjL', '038FZp4Q', '03hLTst4', '03pcT2Nl', '08J5Gwh3', '0CH5VUjd',
       '0FlEud7z', '0GZcBiPN', '0e1gjpDc', '0gmoq98x',
       ...
       'z9SYXq2b', 'zCaHyLJ2', 'zDCufROB', 'zHenMtWq', 'zJWrliL2', 'zKG27hid',
       'zN7t5dpg', 'zVNpdmwh', 'zW0S2mn7', 'zlk0dTGp'],
      dtype='object', name='id', length=1000)

In [146]:
sub = pd.DataFrame({'test_id':frequency_pivot.index,'result':result,'pro':result_pro})

In [147]:
sub.replace({'?mhY5opF':'?mhY5opF4'},inplace=True) 

In [150]:
sub.describe()

,pro
count,1000.000000
mean,0.599604
std,0.246471
min,0.125000
25%,0.384050
50%,0.545455
75%,0.833333
max,1.000000


In [151]:
def threshold(result,distance):
    if distance > 0.545455:
        return result
    else:
        return 

In [152]:
sub['result'] = sub.apply(lambda row: threshold(row['result'],row['pro']),axis=1)

In [153]:
sub.drop('pro',axis=1).to_csv('../output/frequency_threshold50.csv',index=False)

In [149]:
sub.head()

,pro,result,test_id
id,,,
02dDNCjL,0.666667,2Y5w4uGQ,02dDNCjL
038FZp4Q,0.225806,RdQcLlzB,038FZp4Q
03hLTst4,0.888889,iYkr0Rdz,03hLTst4
03pcT2Nl,0.977273,3f8AOLpF,03pcT2Nl
08J5Gwh3,0.382353,0rwxYbXh,08J5Gwh3
